In [1]:
import numpy as np
import pandas as pd
# define stopwords
from nltk.corpus import stopwords
# count vectorizer library
from sklearn.feature_extraction.text import CountVectorizer

# script files
from readfile import readTopic, readTopic2, readDocument, readDocument2
from lda import buildLDA

In [2]:
train_size = []
test_size = []
train_log_likelihood = []
train_perplexity = []
test_log_likelihood = []
test_perplexity = []
final_result = []
final_result_accuracy = []
final_result2 = []
final_result2_accuracy = []
perplexity = []
log_likelihood = []

for x in range(101, 151):
    # read training data
    num = x
    trainSet = readTopic(num)
    train_docs = []
    train_relevant = []

    for set in trainSet:
        lines = readDocument(num, set[0])
        train_docs.append(lines)
        train_relevant.append(int(set[1]))
    
    train_size.append(len(train_docs))
    
    # read testing data
    testSet = readTopic2(num)
    test_docs = []
    test_relevant = []

    for set in testSet:
        lines = readDocument2(num, set[0])
        test_docs.append(lines)
        test_relevant.append(int(set[1]))

    test_size.append(len(test_docs))    
        
    data = {'documents': train_docs, 'relevant': train_relevant}
    df = pd.DataFrame(data = data)




    # NLTK Stop words
    stop_words = stopwords.words('english')

    # build count vectorizer model
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words=stop_words)

    # fit and transform training data
    count = vectorizer.fit_transform(train_docs)
    count_feature_names = vectorizer.get_feature_names()

    # build LDA model
    train_lda_output, train_df_document_topics, train_df, log_likelihood, perplexity = buildLDA(count, train_docs, train_relevant)

    train_log_likelihood.append(log_likelihood)
    train_perplexity.append(perplexity)



    # build count vectorizer model
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words=stop_words)

    # fit training data
    count = vectorizer.fit(train_docs)

    # transform testing data
    count = vectorizer.transform(test_docs)
    count_feature_names = vectorizer.get_feature_names()

    # build LDA model
    test_lda_output, test_df_document_topics, test_df, log_likelihood, perplexity = buildLDA(count, test_docs, test_relevant)

    test_log_likelihood.append(log_likelihood)
    test_perplexity.append(perplexity)



    # calculate the percentage of relevant of each topic
    relevant_counts = []
    irrelevant_counts = []
    percentages = []
    topics = []

    for x in range(0, len(train_df.columns)-2):
        relevant_counts.append(0)
        irrelevant_counts.append(0)

    for x in range(0, len(train_df)):
        if train_df['isRelevant'][x] == 1:
            relevant_counts[train_df['dominant_topic'][x]] += 1
        else:
            irrelevant_counts[train_df['dominant_topic'][x]] += 1

    for x in range(0, len(relevant_counts)):
        if (relevant_counts[x] > 0) and (irrelevant_counts[x] > 0):
            percentage = relevant_counts[x] / (relevant_counts[x] + irrelevant_counts[x])
            percentages.append(percentage)
            if (percentage > 0.5):
                topics.append(x)

    
    # if no topic is more than 50%, use the highest % topic
    if topics == []:
        max_value = max(relevant_counts)
        topics.append(relevant_counts.index(max_value))
    


    # generate the result of testing data
    result = []

    for x in range(0, len(test_df)):
        if test_df['dominant_topic'][x] in topics:
            result.append(1)
        else:
            result.append(0)




    test_result = pd.DataFrame()
    test_result['relevant'] = test_df['isRelevant']
    test_result['countvectorizer'] = result



    correct = 0
    for x in range(0, len(test_result)):
        if test_result['relevant'][x] == test_result['countvectorizer'][x]:
            correct += 1
    print("user: ", num, "        accuracy: ", correct/len(test_result))
    final_result.append(correct)
    final_result_accuracy.append(correct/len(test_result))
    
    
    
    # Get score(d) for each document
    topic_col = []
    for x in topics:
        topic_col.append('Topic' + str(x))

    scores = []
    for index, row in test_df.iterrows():
        score = 0
        for x in topic_col:
            score += row[x]
        scores.append(score)

    test_df['score(d)'] = scores
    df = test_df.sort_values(by=['score(d)'], ascending=False)
    df = df.head(10)
    final_result2.append((df.isRelevant.values == 1).sum())
    final_result2_accuracy.append((df.isRelevant.values == 1).sum() * 0.1)
    
    

Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -6454.97016478401
Model Perplexity:  397.5721552141004
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -59129.519064912645
Model Perplexity:  111.8182846532605
user:  101         accuracy:  0.7590987868284229
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -71625.10000504139
Model Perplexity:  325.34321357113714
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -87160.74363242253
Model Perplexity:  144.86540994149468
user:  102         accuracy:  0.7337662337662337
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -16830.44254859933
Model Perplexity:  421.35117351850494
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -116261.91947439252
Model Perplexity:  195.303094076658
user:

user:  124         accuracy:  0.468
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -10278.51886964253
Model Perplexity:  484.10519413066083
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -84930.91887101243
Model Perplexity:  219.1795107678316
user:  125         accuracy:  0.6911764705882353
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -4978.3509202933055
Model Perplexity:  393.12216705626315
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -28574.15052137032
Model Perplexity:  33.25547512753841
user:  126         accuracy:  0.4888888888888889
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -9271.869979975954
Model Perplexity:  547.5041120215587
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -27608.05795651317
Model 

user:  148         accuracy:  0.5105263157894737
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -5762.4831834708
Model Perplexity:  529.9690024290861
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -59148.558754233956
Model Perplexity:  162.0349400679782
user:  149         accuracy:  0.7906458797327395
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -16831.24446341666
Model Perplexity:  468.75907987544355
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -61262.64020101448
Model Perplexity:  159.51233662469645
user:  150         accuracy:  0.6846361185983828


In [5]:
users = []
for x in range(101, 151):
    users.append(x)
    
df = pd.DataFrame(index=users)
df['train_dataset_size'] = train_size
df['test_dataset_size'] = test_size
df['total_correct'] = final_result
df['total_accuracy'] = final_result_accuracy
df['top10_correct'] = final_result2
df['top10_accuracy'] = final_result2_accuracy
df['train_log_likelihood'] = train_log_likelihood
df['train_perplexity'] = train_perplexity
df['test_log_likelihood'] = test_log_likelihood
df['test_perplexity'] = test_perplexity
df

,train_dataset_size,test_dataset_size,total_correct,total_accuracy,top10_correct,top10_accuracy,train_log_likelihood,train_perplexity,test_log_likelihood,test_perplexity
101,23,577,438,0.759099,8,0.8,-6454.970165,397.572155,-59129.519065,111.818285
102,199,308,226,0.733766,7,0.7,-71625.100005,325.343214,-87160.743632,144.865410
103,64,528,252,0.477273,1,0.1,-16830.442549,421.351174,-116261.919474,195.303094
104,194,279,169,0.605735,7,0.7,-66199.399670,218.773294,-75876.558782,123.554967
105,37,258,207,0.802326,5,0.5,-27399.304497,82.689146,-37465.531825,77.676606
106,44,321,239,0.744548,0,0.0,-11823.344747,340.514451,-73326.784362,172.340021
107,61,571,283,0.495622,0,0.0,-29544.889481,402.760438,-105976.650178,117.073468
108,53,386,215,0.556995,0,0.0,-13757.814216,641.289441,-54627.496179,244.699550
109,40,240,120,0.500000,0,0.0,-13609.911546,606.334635,-43935.829117,335.284365
110,91,491,417,0.849287,0,0.0,-28859.222651,453.947269,-113194.566844,323.737838


In [14]:
# path=r'C:\Users\sumsu\Desktop\QUT\sem2\IFN712\project\RCV1 Dataset'
# df.to_csv(path+'countvectorizer_result.csv')
df.to_csv('countvectorizer_result.csv')